# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import re
import pickle
import pandas as pd 
from sqlalchemy import create_engine
import nltk 
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split,  GridSearchCV 
from sklearn.metrics import classification_report


nltk.download(['wordnet', 'punkt', 'stopwords'])

In [ ]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('disaster_messages', 'sqlite:///InsertDatabaseName.db')
X = df['message']  # Message Column
Y = df.iloc[:, 4:] # Classification label

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    """
    Function: split text into words and return the root form of the words
    Args:
      text(str): the message
    Return:
      lemm(list of str): a list of the root form of the message words
    """
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text
    words = word_tokenize(text)
    
    # Remove stop words
    stop = stopwords.words("english")
    words = [t for t in words if t not in stop]
    
    # Lemmatization
    lemm = [WordNetLemmatizer().lemmatize(w) for w in words]
    return lemm

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
# Pipleine 1: Random Forest Classifier

pipeline_rfc = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(RandomForestClassifier()))
    ])


# Pipeline 2: Naive Bayes classifier

pipeline_nbc = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(MultinomialNB()))
    ])


# Pipleine 3: Adaboost Classifier 

pipeline_ada = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
# Spiliting data
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [ ]:
# Fit the Random Forest Classifier 
pipeline_rfc.fit(X_train, y_train)

In [ ]:
# Fit the Naive Bayes classifier 
pipeline_nbc.fit(X_train, y_train)

In [ ]:
# Fit the Adaboost Classifier 
pipeline_ada.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
def plot_scores(y_test, y_pred):
    #Testing the model
    # Printing the classification report for each label
    i = 0
    for col in y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(y_test[col], y_pred[:, i]))
        i = i + 1
    accuracy = (y_pred == y_test.values).mean()
    print('The model accuracy is {:.3f}'.format(accuracy))

In [ ]:
# Prediction: the Random Forest Classifier  
y_pred = pipeline_rfc.predict(X_test)
plot_scores(y_test, y_pred)

In [ ]:
# Prediction: the Naive Bayes classifier 
y_pred = pipeline_nbc.predict(X_test)
plot_scores(y_test, y_pred)

In [ ]:
# Prediction: the Adaboost Classifier 
y_pred = pipeline_ada.predict(X_test)
plot_scores(y_test, y_pred)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# Show parameters for the pipline
pipeline_rfc.get_params()

In [ ]:
# Show parameters for the pipline
pipeline_ada.get_params()

In [ ]:
# Using grid search
# Create Grid search parameters for Random Forest Classifier   
parameters_rfc = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [10, 20]
}

cv_rfc = GridSearchCV(pipeline_rfc, param_grid = parameters_rfc)
cv_rfc 

In [ ]:
# Using grid search
# Create Grid search parameters
parameters_ada = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 60, 70]
}

cv_ada = GridSearchCV(pipeline_ada, param_grid = parameters_ada)
cv_ada

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# Fit the first tuned model
cv_rfc.fit(X_train, y_train)

In [ ]:
# Fit the second tuned model
cv_ada.fit(X_train, y_train)

In [ ]:
# Predicting using the first tuned model 
y_pred = cv_rfc.predict(X_test)
plot_scores(y_test, y_pred)

In [ ]:
# Predicting using the second tuned model 
y_pred = cv_ada.predict(X_test)
plot_scores(y_test, y_pred)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
# Create a pickle file for the model
file_name = 'classifier.pkl'
with open (file_name, 'wb') as f:
    pickle.dump(cv_ada, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.